# Двоичное дерево
В двоичном дереве поиска:
 - каждая вершина может иметь (или не иметь) левого и правого ребенка;
 - каждая вершина, кроме корня, имеет родителя.
 
Для каждой вершины дерева мы храним:
- значение value
- левый ребенок left
- правый ребенок right
- родитель parent

Ключи в двоичном дереве поиска хранятся с соблюдением свойства упорядоченности:

Пусть $x$ - произвольная вершина двоичного дерева поиска. Если вершина $y$ находится в левом поддереве вершины $x$, то $y.key \ge x.key$ . Если $y$ находится в правом поддереве x, то $y.key \le x.key$ .

In [43]:
import random
from binarytree import build

# класс узла дерева
class MyNode:
    def __init__(self,value,parent=None,left=None,right=None):
        self.value = value
        self.parent = parent
        self.right = right
        self.left = left

    def __repr__(self):
        return str(self.__dict__)

# класс двоичного дерева поиска 
class BSTree():
    def __init__(self):
        self.root = None
        self.data = {}

    def __repr__(self):
        res = "root="+str(self.root)+"\n"
        for k,v in self.data.items():
            res += f'{k} => {v} \n'
        return res
    
    def find_by_key(self, key):
        if key in self.data.keys(): return self.data[key].value
        else: return None
        
    def __add_node(self,value,parent=None,left=None,right=None):
        # имитируем указатели для питона - генерируем уникальный id для каждого узла дерева        
        id = random.randint(0,10**6)
        while id in self.data.keys():
            id = random.randint(0,2**31-1)
        self.data[id] = MyNode(value,parent,left,right)
        return id

    def __delete_node(self,node):
        return self.data.pop(node, None)

    def insert(self,value):
        self.root = self.__insert(value,self.root, self.root)

    def __insert(self,value, root = None, parent=None):
        if root == None:
            return self.__add_node(value,parent,None,None)
        elif value < self.data[root].value:
            self.data[root].left = self.__insert(value, self.data[root].left, root)
        elif value > self.data[root].value:
            self.data[root].right = self.__insert(value, self.data[root].right, root)
        return root

    def print(self):
        print(self.__print(self.root,' '))

    def __print(self,root,deep):
        res = deep + str(self.data[root].value)+f' ({root})' + '\n'
        if self.data[root].left != None:
            res +=self.__print(self.data[root].left, deep+'-')
        if self.data[root].right != None:
            res += self.__print(self.data[root].right,deep+'+')
        return res

    def search(self,value):
        return self.__search(value,self.root)

    def __search(self,value,root):
        if root == None or value == self.data[root].value:
            return root
        if value < self.data[root].value:
            return self.__search(value,self.data[root].left)
        else:
            return self.__search(value,self.data[root].right)

    def minimum(self,return_index=False):
        return self.__minimum(self.root,return_index)

    def __minimum(self, root,return_index = False):
        if self.data[root].left == None:
            if return_index:
                return root
            else:
                return self.data[root].value
        else:
            return self.__minimum(self.data[root].left,return_index)

    def maximum(self,return_index=False):
        return self.__maximum(self.root,return_index)

    def __maximum(self, root,return_index = False):
        if self.data[root].right == None:
            if return_index:
                return root
            else:
                return self.data[root].value
        else:
            return self.__maximum(self.data[root].right,return_index)
    
    # поиск узла по значению    
    def find_node_by_value(self,value):
        res = list(key for key,node in self.data.items() if node.value==value)
        if len(res)!=1:
            return None
        else:
            return res[0]

    # горизонтальный обход с помощью очереди
    def print_by_levels(self):
        res = []
        top = self.root
        queue = []
        while True:
            res.append(self.data[top].value)
            if self.data[top].left != None:
                queue.append(self.data[top].left)
            if self.data[top].right != None:
                queue.append(self.data[top].right)
            if len(queue)>0:
                top = queue.pop(0)
            else:
                break
        return res

    # достраивание дерева до полного, заполнение X пустых ячеек. 
    # Это необходимо для визуализации с помощью binarytree
    def print_by_levels_full(self):
        height = self.height()
        # создаем копию текущего дерева
        mystic = BSTree()
        for node in self.data.values():
            mystic.insert(node.value)
        # достраиваем его до полного None узлами
        mystic.__make_full_tree(mystic.root,height)
        # получаем его запись в горизонтальном обходе. Эта запись пригодна
        # для визуализации binary tree
        full = mystic.print_by_levels()
        return full

    # понятный вывод дерево с помощью сторонней библиотеки
    def print_nice(self):
        if len(self.data)>0:
            full = self.print_by_levels_full()
        else:
            full = [None]
        bt = build(full)
        print(bt)
        return None


    # создание узлов со значением None, чтобы дерево было полным
    def __make_full_tree(self,root,height):
        if height>0:
            if self.data[root].left == None:
                self.data[root].left = self.__add_node(None,parent=root,left=None,right=None)
            if height > 1:
                self.__make_full_tree(self.data[root].left, height-1)
            if self.data[root].right == None:
                self.data[root].right = self.__add_node(None,parent=root,left=None,right=None)
            if height > 1:
                self.__make_full_tree(self.data[root].right, height-1)

    # поиск следующего по ключу узла дерева     
    # время работы O(h), где h - высота дерева
    def find_next(self,node):
        if node not in self.data.keys(): return None
        if self.data[node].right != None:
            return self.__minimum(self.data[node].right,return_index=True)
        else:
            # идем наверх налево, пока не получится свернуть направо
            top_node = self.data[node].parent
            while top_node!= None and node == self.data[top_node].right:
                node = top_node
                top_node = self.data[top_node].parent
            return top_node

    # удаление элемента с одним и меньше ребенком = просто возвращаем этого ребенка или none
    def __remove1child(self,node):
        if self.data[node].left == None and self.data[node].right != None:
            # return self.__delete_node(node).right
            self.data[self.data[node].right].parent=self.data[node].parent
            return self.data[node].right
        if self.data[node].right == None and self.data[node].left != None:
            # return self.__delete_node(node).left
            self.data[self.data[node].left].parent = self.data[node].parent
            return self.data[node].left
        if self.data[node].right == None and self.data[node].left == None:
            # self.__delete_node(node)
            return None

    # удаление элемента с двумя детьми - на место удаляемого сдвигаем минимальный справа
    def __remove2child(self,node):
        # находим минимальный элемент справа
        min_index = self.__minimum(self.data[node].right, return_index=True)
        # удаляем этот минимальный элемент
        self.data[node].right = self.__remove(min_index,self.data[node].right)
        # ставим на место удаляемого элемента минимальный элемент
        self.data[min_index].right = self.data[node].right
        self.data[min_index].left = self.data[node].left
        # self.__delete_node(node)
        return min_index


    def __remove(self, node, root):
        # идем в левое поддерево
        if self.data[root].value>self.data[node].value:
            self.data[root].left = self.__remove(node, self.data[root].left)
            return root
        # идем в правое поддерево
        elif self.data[root].value<self.data[node].value:
            self.data[root].right = self.__remove(node, self.data[root].right)
            return root
        # уже корень поддерева совпадает с удаляемым элементом
        else:
            if self.data[node].left == None or self.data[node].right == None:
                return self.__remove1child(node)
            else:
                return self.__remove2child(node)

    # удаляем узел node_index, перестраиваем дерево
    def remove(self, node_index):
        if node_index not in self.data.keys():
            return None
        self.root = self.__remove(node_index,self.root)
        self.__delete_node(node_index)

    def height(self):
        if self.root==None:
            return 0
        elif len(self.data) == 1:
            return 1
        else:
            return self.__height(self.root)

    def __height(self,root):
        if self.data[root].left == None and self.data[root].right == None:
            return 0
        else:
            left_height, right_height = 0,0
            if self.data[root].left!=None:
                left_height = self.__height(self.data[root].left)
            if self.data[root].right!=None:
                right_height = self.__height(self.data[root].right)
            if left_height>right_height:
                return left_height+1
            else:
                return right_height+1

    # удаляет поддерево со значением value
    def remove_subtree_by_value(self, value):
        self.__remove_subtree_by_value(value,self.root)

    def __remove_subtree_by_value(self, value, root):
        if self.data[root].value < value:
            self.data[root].right = self.__remove_subtree_by_value(value,self.data[root].right)
            return root
        elif self.data[root].value > value:
            self.data[root].left = self.__remove_subtree_by_value(value,self.data[root].left)
            return root
        else:
            self.__remove_subtree_by_node(root)
            # возвращаем None, чтобы уничтожить ссылку на себя в left/right родительского узла
            return None
        pass

    # удаляем поддерево по id узла
    def __remove_subtree_by_node(self, root):
        if self.data[root].left != None:
            self.__remove_subtree_by_node(self.data[root].left)
        if self.data[root].right != None:
            self.__remove_subtree_by_node(self.data[root].right)
        self.__delete_node(root)

In [44]:
print('Продемонстрируем двоичное дерево поиска из n элементов.')
b = BSTree()
# n=random.randint(1,20)
n = 10
values = [i for i in range(0,n)]
random.shuffle(values)
# values = [0, 2, 6, 4, 5, 8, 14, 7, 9, 1, 12, 3, 10, 13, 11]
print('Случайный массив чисел, они же - ключи дерева')
print(values)
for val in values:
    b.insert(val)
print('Строим дерево поиска из этих элементов.')
b.print_nice()

Продемонстрируем двоичное дерево поиска из n элементов.
Случайный массив чисел, они же - ключи дерева
[6, 7, 3, 9, 0, 5, 4, 1, 2, 8]
Строим дерево поиска из этих элементов.

        ____6
       /     \
  ____3__     7__
 /       \       \
0         5       9
 \       /       /
  1     4       8
   \
    2



In [45]:
print('Продемонстрируем нахождение высоты дерева поиска.')
print(b.height())


Продемонстрируем нахождение высоты дерева поиска.
4


In [46]:
print('Продемонстрируем нахождение минимального элемента в дереве поиска.')
print(b.minimum())
# print(b.minimum(return_index=True))
print('Продемонстрируем нахождение максимального элемента в дереве поиска.')
print(b.maximum())
# print(b.maximum(return_index=True))

Продемонстрируем нахождение минимального элемента в дереве поиска.
0
Продемонстрируем нахождение максимального элемента в дереве поиска.
9


In [47]:
print('Продемонстрируем нахождение следующего за заданным элементом в дереве поиска.')
element = input('Введите элемент, следующий за которым нужно найти')
element_id = b.search(int(element))
print(b.find_by_key(b.find_next(element_id)))

Продемонстрируем нахождение следующего за заданным элементом в дереве поиска.
Введите элемент, следующий за которым нужно найти4
5


In [ ]:
print('Продемонстрируем удаление поддеревьев из дерева поиска.',
      'Если удаляем корень дерева, то все дерево исчезает.',
      'Вводите ключ корян поддерева или n, чтобы закончить процесс.')

b = BSTree()
n = 10
values = [i for i in range(0,n)]
random.shuffle(values)
for val in values:
    b.insert(val)
print('Строим дерево поиска из этих элементов.')
b.print_nice()

command = input('Введите номер корня поддерева для удаления либо n?')
while command != 'n':
    b.remove_subtree_by_value(int(command))
    print('Мы удалили ', command)
    b.print_nice()
    command = input('Введите номер корня поддерева для удаления либо n?')

Продемонстрируем удаление поддеревьев из дерева поиска. Если удаляем корень дерева, то все дерево исчезает. Вводите ключ корян поддерева или n, чтобы закончить процесс.
Строим дерево поиска из этих элементов.

    __3__________
   /             \
  1       ________9
 / \     /
0   2   4______
               \
            ____8
           /
          5
           \
            6
             \
              7

Введите номер корня поддерева для удаления либо n?4
Мы удалили  4

    __3
   /   \
  1     9
 / \
0   2

Введите номер корня поддерева для удаления либо n?1
Мы удалили  1

3
 \
  9

Введите номер корня поддерева для удаления либо n?3
Мы удалили  3
None
